In [1]:
import sys
import pandas as pd
import numpy as np

%pip install textblob
from textblob import TextBlob

import nltk
nltk.download('punkt')

Note: you may need to restart the kernel to use updated packages.


[nltk_data] Downloading package punkt to C:\Users\Ville
[nltk_data]     Kylmämaa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
reviews = pd.read_csv('Restaurant_Reviews.tsv', sep='\t')
print(reviews)

                                                Review  Liked
0                             Wow... Loved this place.      1
1                                   Crust is not good.      0
2            Not tasty and the texture was just nasty.      0
3    Stopped by during the late May bank holiday of...      1
4    The selection on the menu was great and so wer...      1
..                                                 ...    ...
995  I think food should have flavor and texture an...      0
996                           Appetite instantly gone.      0
997  Overall I was not impressed and would not go b...      0
998  The whole experience was underwhelming, and I ...      0
999  Then, as if I hadn't wasted enough of my life ...      0

[1000 rows x 2 columns]


In [3]:
reviews_column = reviews["Review"]
blob = TextBlob(reviews_column.to_string(index=False))
print(blob)

                          Wow... Loved this place.
                                Crust is not good.
         Not tasty and the texture was just nasty.
Stopped by during the late May bank holiday off...
The selection on the menu was great and so were...
    Now I am getting angry and I want my damn pho.
             Honeslty it didn't taste THAT fresh.)
The potatoes were like rubber and you could tel...
                         The fries were great too.
                                    A great touch.
                          Service was very prompt.
                                Would not go back.
The cashier had no care what so ever on what I ...
I tried the Cape Cod ravoli, chicken, with cran...
I was disgusted because I was pretty sure that ...
I was shocked because no signs indicate cash only.
                               Highly recommended.
            Waitress was a little slow in service.
This place is not worth your time, let alone Ve...
                              d

In [4]:
sentiment_polarity = []
liked_column = reviews["Liked"]

for sentence in blob.sentences:
    sentiment_polarity.append(sentence.sentiment.polarity)

# TODO miksei toimi?
print(np.corrcoef(sentiment_polarity, liked_column))

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 791 and the array at index 1 has size 1000